In [1]:
!pip install datasets transformers evaluate langchain faiss-cpu langchain-community langchain_openai rouge_score

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.5 MB/s eta 0:00:00
   ━━━━

In [3]:
import time
import numpy as np
import pandas as pd
import evaluate
from transformers import pipeline
from datasets import load_dataset
from datasets import concatenate_datasets
from langchain.schema import Document
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from google.api_core.exceptions import ResourceExhausted



In [4]:
full_val = load_dataset("squad")["validation"]
full_train = load_dataset("squad")["train"]

in_domain = full_val.shuffle(seed=0).select(range(10))
out_domain = full_train.shuffle(seed=1).select(range(10))
eval_ds = concatenate_datasets([in_domain, out_domain])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [5]:
docs = [Document(page_content=ex["context"], metadata={"id": str(i)})
        for i, ex in enumerate(in_domain)]
emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(docs, emb)


<ipython-input-5-2f01a2eb6bd3>:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  emb = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
# Extractive
ext_qa = pipeline("question-answering",
                  model="huggingface-course/bert-finetuned-squad",
                  tokenizer="huggingface-course/bert-finetuned-squad",
                  device=-1)

config.json:   0%|          | 0.00/686 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/320 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/431M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [7]:
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

In [8]:
# Generative
gen_llm = ChatOpenAI(model="gpt-4o", max_tokens=64, temperature=0.0,api_key=api_key)
gen_chain = LLMChain(
    llm=gen_llm,
    prompt=PromptTemplate(
        input_variables=["question"],
        template="Answer the question: {question}"
    )
)




<ipython-input-8-47b5f21e7889>:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  gen_chain = LLMChain(


In [9]:
# RAG
rag_qa = RetrievalQA.from_chain_type(
    llm=gen_llm,
    chain_type="stuff",
    retriever=vectorstore.as_retriever(k=5),
)


In [10]:
squad_metric = evaluate.load("squad")
rouge      = evaluate.load("rouge")

In [11]:
def retrieval_stats(question, gold_idx):
    docs = vectorstore.as_retriever(k=5).get_relevant_documents(question)
    ids  = [int(d.metadata["id"]) for d in docs]
    # recall@5
    recall = 1.0 if gold_idx in ids else 0.0
    # mrr
    if gold_idx in ids:
        rank = ids.index(gold_idx) + 1
        mrr  = 1.0 / rank
    else:
        mrr = 0.0
    return recall, mrr



In [12]:
results = []
for approach, fn in [
    ("Extractive", lambda q, c, i: ext_qa(question=q, context=c)["answer"]),
    ("Generative", lambda q, c, i: gen_chain.invoke({"question": q})["text"].strip()),
    ("RAG",        lambda q, c, i: rag_qa.run(q).strip())
]:
    preds, refs = [], []
    recalls, mrrs = [], []
    times = []

    start_time = time.time()  # Start timing for the approach

    for i, ex in enumerate(eval_ds):
        q, ctx = ex["question"], ex["context"]
        golds   = ex["answers"]["text"]
        try:
            ans = fn(q, ctx, i if i < 10 else None)
        except ResourceExhausted:
            ans = fn(q, ctx, i if i < 10 else None)

        preds.append({"id": str(i), "prediction_text": ans})
        refs.append({"id": str(i), "answers": {"text": golds, "answer_start": ex["answers"]["answer_start"]}})

        # only compute retrieval metrics for in-domain (i < 10)
        if approach == "RAG" and i < 10:
            r, m = retrieval_stats(q, i)
        else:
            r, m = None, None
        recalls.append(r)
        mrrs.append(m)

    end_time = time.time()  # End timing for the approach
    elapsed_time = end_time - start_time  # Total time taken
    qps = len(eval_ds) / elapsed_time  # Calculate queries per second (QPS)

    # core QA metrics
    scores = squad_metric.compute(predictions=preds, references=refs)
    # rouge
    rouge_scores = rouge.compute(predictions=[p["prediction_text"] for p in preds],
                                 references=[r["answers"]["text"][0] for r in refs])

    results.append({
        "Approach": approach,
        "EM": scores["exact_match"],
        "F1": scores["f1"],
        "ROUGE-L": rouge_scores["rougeL"],
        "Recall@5 (in-d)": np.nanmean([r for r in recalls if r is not None]),
        "MRR@5 (in-d)":    np.nanmean([m for m in mrrs    if m is not None]),
        "Throughput (QPS)": round(qps, 2)
    })

<ipython-input-12-05a1db592781>:47: RuntimeWarning: Mean of empty slice
  "Recall@5 (in-d)": np.nanmean([r for r in recalls if r is not None]),
<ipython-input-12-05a1db592781>:48: RuntimeWarning: Mean of empty slice
  "MRR@5 (in-d)":    np.nanmean([m for m in mrrs    if m is not None]),
<ipython-input-12-05a1db592781>:47: RuntimeWarning: Mean of empty slice
  "Recall@5 (in-d)": np.nanmean([r for r in recalls if r is not None]),
<ipython-input-12-05a1db592781>:48: RuntimeWarning: Mean of empty slice
  "MRR@5 (in-d)":    np.nanmean([m for m in mrrs    if m is not None]),
<ipython-input-12-05a1db592781>:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  ("RAG",        lambda q, c, i: rag_qa.run(q).strip())
<ipython-input-11-5e0451513a29>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :met

In [13]:
df = pd.DataFrame(results)
df.set_index("Approach", inplace=True)
df


,EM,F1,ROUGE-L,Recall@5 (in-d),MRR@5 (in-d),Throughput (QPS)
Approach,,,,,,
Extractive,90.0,96.285714,0.933333,NaN,NaN,2.30
Generative,0.0,5.960180,0.054055,NaN,NaN,0.54
RAG,0.0,13.926623,0.112635,1.0,1.0,1.21
